<a href="https://colab.research.google.com/github/Palaeoprot/ModulAAR/blob/main/main_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install nbimporter

In [ ]:
# prompt: authenticate colab OAuth client c

from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import nbformat
from IPython import get_ipython
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from datetime import datetime

R = 8.314  # Gas constant in J/(mol·K)

def execute_notebook(notebook_path):
    """Execute a Jupyter notebook and return its namespace."""
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    namespace = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            exec(cell.source, namespace)
    return namespace

# Functions from hydrolysis_decomposition_estimates.ipynb
def fit_decomposition_rate(data, temperatures, times):
    """Fit the decomposition rate based on the total loss of amino acids, assuming second-order reaction."""
    def objective(k_d):
        residuals = []
        for temp in temperatures:
            temp_data = data[data['temp (°C)'] == temp]
            for t in times:
                observed_THAA = temp_data[temp_data['time'] == t]['THAA_total'].values
                if len(observed_THAA) == 0:
                    continue
                A0 = observed_THAA[0]
                predicted_THAA = A0 / (1 + k_d * A0 * t)
                residuals.append(predicted_THAA - observed_THAA[0])
        return np.array(residuals).flatten()

    initial_k_d = 1e-7
    result = least_squares(objective, initial_k_d, bounds=(0, np.inf))
    return result.x[0]

def fit_hydrolysis_rate(data, temperatures, times, k_d):
    """Fit the hydrolysis rate after adjusting for decomposition, assuming second-order reaction."""
    def objective(k_h):
        residuals = []
        for temp in temperatures:
            temp_data = data[data['temp (°C)'] == temp]
            for t in times:
                observed_FAA = temp_data[temp_data['time'] == t]['FAA_total'].values
                if len(observed_FAA) == 0:
                    continue
                A0 = observed_FAA[0]
                predicted_FAA = A0 / (1 + k_h * A0 * t)
                residuals.append(predicted_FAA - observed_FAA[0])
        return np.array(residuals).flatten()

    initial_k_h = 1e-6
    result = least_squares(objective, initial_k_h, bounds=(0, np.inf))
    return result.x[0]

def scale_to_reference_temp(data, ref_temp, activation_energy=98000):
    """Scale data to a reference temperature using the Arrhenius equation."""
    ref_temp_K = ref_temp + 273.15
    scaled_data = data.copy()

    for index, row in data.iterrows():
        temp_K = row['temp (°C)'] + 273.15
        scaling_factor = np.exp(-activation_energy * (1 / ref_temp_K - 1 / temp_K) / R)
        scaled_data.at[index, 'time'] *= scaling_factor

    return scaled_data

def optimise_rates(data):
    """Optimize hydrolysis and decomposition rates for each pH level separately."""
    print("Optimise_rates function called")  # Debug print
    pH_levels = data['pH'].unique()
    optimized_parameters = []

    for pH in pH_levels:
        pH_data = data[data['pH'] == pH]
        temperatures = pH_data['temp (°C)'].unique()
        times = pH_data['time'].unique()

        print(f"Optimizing for pH: {pH}")
        print(f"Temperatures: {temperatures}")
        print(f"Times: {times}")

        k_d_opt = fit_decomposition_rate(pH_data, temperatures, times)
        print(f"Fitted decomposition rate (k_d) for pH {pH}: {k_d_opt}")

        k_h_opt = fit_hydrolysis_rate(pH_data, temperatures, times, k_d_opt)
        print(f"Fitted hydrolysis rate (k_h) for pH {pH}: {k_h_opt}")

        optimized_parameters.append((pH, k_h_opt, k_d_opt))

    return optimized_parameters

def plot_results(data, optimized_parameters, ref_temp=110):
    """Plot observed and predicted THAA and FAA concentrations for each pH level."""
    print("Plot_results function called")  # Debug print
    scaled_data = scale_to_reference_temp(data, ref_temp)
    pH_levels = scaled_data['pH'].unique()
    temperature_symbols = {80: 'o', 110: '^', 140: 's'}
    pH_colors = {5: 'r', 7: 'g', 9: 'b'}

    plt.figure(figsize=(14, 6))

    for pH, k_h, k_d in optimized_parameters:
        pH_data = scaled_data[scaled_data['pH'] == pH]
        temperatures = pH_data['temp (°C)'].unique()

        for temp in temperatures:
            T = temp + 273.15
            temp_data = pH_data[pH_data['temp (°C)'] == temp]
            observed_THAA = temp_data['THAA_total']
            observed_FAA = temp_data['FAA_total']
            time = temp_data['time']

            predicted_THAA = observed_THAA / (1 + k_d * observed_THAA * time)
            predicted_FAA = observed_FAA / (1 + k_h * observed_FAA * time)

            plt.subplot(1, 2, 1)
            plt.plot(time, observed_THAA, temperature_symbols[temp], color=pH_colors[pH], label=f'Observed THAA pH {pH}, {temp}°C')
            plt.plot(time, predicted_THAA, '-', color=pH_colors[pH], label=f'Predicted THAA pH {pH}, {temp}°C')

            plt.subplot(1, 2, 2)
            plt.plot(time, observed_FAA, temperature_symbols[temp], color=pH_colors[pH], label=f'Observed FAA pH {pH}, {temp}°C')
            plt.plot(time, predicted_FAA, '-', color=pH_colors[pH], label=f'Predicted FAA pH {pH}, {temp}°C')

    plt.subplot(1, 2, 1)
    plt.xscale('log')
    plt.xlabel('Time (hours)')
    plt.ylabel('THAA_total')
    plt.title(f'THAA over Time scaled to {ref_temp}°C')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.xscale('log')
    plt.xlabel('Time (hours)')
    plt.ylabel('FAA_total')
    plt.title(f'FAA over Time scaled to {ref_temp}°C')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Rest of your functions (load_data, write_data, calculate_relative_rates, prepare_data_for_google_sheets) remain unchanged

def main():
    # Load the aggregated data
    data_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ProcessedData/real_DL_summed_totals.csv'
    data = pd.read_csv(data_path)

    # Check if data is loaded correctly
    if data.empty:
        print("Data is empty. Please check the data file.")
    else:
        print("Data loaded successfully.")
        print(data.head())

    # Optimize rates
    try:
        optimized_parameters = optimise_rates(data)

        # Prepare data for Google Sheets
        rows = prepare_data_for_google_sheets(optimized_parameters)

        # Append data to Google Sheets
        sheet_id = '1nA6jSAkAf1Ud-kHdaYTMtBPgKhe9nBg_IjM9idLlj8E'
        gid = '1475619581'
        write_data(sheet_id, gid, rows)

        # Plot the results
        plot_results(data, optimized_parameters)

        # Arrhenius plots
        temperatures = data['temp (°C)'].unique()
        k_h_values = [params[1] for params in optimized_parameters]
        k_d_values = [params[2] for params in optimized_parameters]
        arrhenius_plot(temperatures, k_h_values, 'Arrhenius Plot for Hydrolysis', 'ln(k_h)')
        arrhenius_plot(temperatures, k_d_values, 'Arrhenius Plot for Decomposition', 'ln(k_d)')

    except Exception as e:
        print(f"Failed to optimize rates or plot results: {str(e)}")

def arrhenius_plot(temps, rate_constants, title, ylabel):
    """Generate an Arrhenius plot."""
    inverse_temps = 1 / (temps + 273.15)  # Convert to Kelvin and take the inverse
    log_rate_constants = np.log(rate_constants)

    plt.figure()
    plt.plot(inverse_temps, log_rate_constants, 'o')
    plt.xlabel('1 / Temperature (1/K)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

if __name__ == '__main__':
    main()

Data loaded successfully.
   temp (°C)   time   pH    BAA_total   FAA_total    THAA_total
0       80.0    0.0  5.0  9774.715478  225.284522  10000.000000
1       80.0   24.0  5.0  7015.646610  129.305317   7144.951927
2       80.0  120.0  5.0  9774.715478  225.284522  10000.000000
3       80.0  480.0  5.0  8210.887641  332.761235   8543.648876
4       80.0  960.0  5.0  6166.497259  791.828437   6958.325696
Optimise_rates function called
Optimizing for pH: 5.0
Temperatures: [ 80. 110. 140.]
Times: [0.00e+00 2.40e+01 1.20e+02 4.80e+02 9.60e+02 1.44e+03 2.40e+02 3.84e+02
 1.00e+00 2.00e+00 6.00e+00 7.20e+01 9.60e+01]
Fitted decomposition rate (k_d) for pH 5.0: 3.7579831056820815e-16
Fitted hydrolysis rate (k_h) for pH 5.0: 1.443740386774812e-14
Optimizing for pH: 7.0
Temperatures: [ 80. 110. 140.]
Times: [0.0000e+00 2.4000e+01 9.7300e+01 1.2000e+02 4.8000e+02 7.2000e+02
 9.6000e+02 1.4400e+03 2.1601e+03 4.0288e+03 5.8790e+03 2.4000e+02
 3.8400e+02 8.4000e+02 1.2000e+03 1.0000e+00 2.0000e+

In [ ]:
import nbformat
from IPython import get_ipython
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from datetime import datetime

R = 8.314  # Gas constant in J/(mol·K)

def execute_notebook(notebook_path):
    """Execute a Jupyter notebook and run its code cells."""
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    ip = get_ipython()
    global_namespace = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                ip.run_cell(cell.source)
            except Exception as e:
                print(f"Error executing cell: {e}")

    return global_namespace

def load_data(sheet_id, gid):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ModulAAR_Credentials/client_secret_764535429800-5tqdene2ic76kkt3k31d10ordk1s20kk.apps.googleusercontent.com.json',
        scope
    )
    client = gspread.authorize(creds)

    sheet = client.open_by_key(sheet_id)
    worksheet = sheet.get_worksheet_by_id(gid)
    data = worksheet.get_all_records()

    return pd.DataFrame(data)

def write_data(sheet_id, gid, rows):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ModulAAR_Credentials/client_secret_764535429800-5tqdene2ic76kkt3k31d10ordk1s20kk.apps.googleusercontent.com.json',
        scope
    )
    client = gspread.authorize(creds)

    sheet = client.open_by_key(sheet_id)
    worksheet = sheet.get_worksheet_by_id(gid)

    for row in rows:
        worksheet.append_row(row)

    print("Data successfully appended to Google Sheets.")

def calculate_relative_rates(optimized_parameters, middle_temp=110, middle_pH=7):
    relative_rates = []
    reference_rate = None

    # Find the reference rate (for middle temperature and pH)
    for pH, k_h, k_d in optimized_parameters:
        if pH == middle_pH:
            reference_rate = (middle_temp, pH, k_h, k_d)
            break

    if not reference_rate:
        raise ValueError("Reference rate not found. Please check the input parameters.")

    ref_temp, ref_pH, ref_k_h, ref_k_d = reference_rate

    for pH, k_h, k_d in optimized_parameters:
        relative_k_h = k_h / ref_k_h
        relative_k_d = k_d / ref_k_d
        relative_rates.append((middle_temp, pH, k_h, k_d, relative_k_h, relative_k_d))

    return relative_rates

def prepare_data_for_google_sheets(optimized_parameters):
    relative_rates = calculate_relative_rates(optimized_parameters)
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Prepare rows to append to Google Sheets
    rows = []
    for temp, pH, k_h, k_d, relative_k_h, relative_k_d in relative_rates:
        rows.append([current_date, temp, pH, k_h, k_d, relative_k_h, relative_k_d])

    return rows

def main():
    # Path to the hydrolysis and decomposition estimates notebook
    notebook_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/hydrolysis_decomposition_estimates.ipynb'

    # Execute the hydrolysis_decomposition_estimates notebook
    try:
        print(f"Executing {notebook_path}...")
        global_namespace = execute_notebook(notebook_path)
        print(f"Successfully executed {notebook_path}")
    except Exception as e:
        print(f"Failed to execute {notebook_path}: {str(e)}")

    # Verify if the functions are defined
    try:
        optimise_rates = global_namespace['optimise_rates']
        plot_results = global_namespace['plot_results']
        print("Successfully accessed functions from hydrolysis_decomposition_estimates notebook")
    except KeyError:
        print("Failed to access functions - they may not be defined.")
        optimise_rates = None
        plot_results = None

    if optimise_rates and plot_results:
        # Load the aggregated data
        data_path = '/content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/ProcessedData/real_DL_summed_totals.csv'
        data = pd.read_csv(data_path)

        # Check if data is loaded correctly
        if data.empty:
            print("Data is empty. Please check the data file.")
        else:
            print("Data loaded successfully.")
            print(data.head())

        # Optimize rates
        try:
            optimized_parameters = optimise_rates(data)

            # Prepare data for Google Sheets
            rows = prepare_data_for_google_sheets(optimized_parameters)

            # Append data to Google Sheets
            sheet_id = '1nA6jSAkAf1Ud-kHdaYTMtBPgKhe9nBg_IjM9idLlj8E'
            gid = '1475619581'
            write_data(sheet_id, gid, rows)

            # Plot the results
            plot_results(data, optimized_parameters)

            # Arrhenius plots
            temperatures = data['temp (°C)'].unique()
            k_h_values = [params[1] for params in optimized_parameters]
            k_d_values = [params[2] for params in optimized_parameters]
            arrhenius_plot(temperatures, k_h_values, 'Arrhenius Plot for Hydrolysis', 'ln(k_h)')
            arrhenius_plot(temperatures, k_d_values, 'Arrhenius Plot for Decomposition', 'ln(k_d)')

        except Exception as e:
            print(f"Failed to optimize rates or plot results: {str(e)}")
    else:
        print("Skipping optimization and plotting due to function access issues.")

def arrhenius_plot(temps, rate_constants, title, ylabel):
    """Generate an Arrhenius plot."""
    inverse_temps = 1 / (temps + 273.15)  # Convert to Kelvin and take the inverse
    log_rate_constants = np.log(rate_constants)

    plt.figure()
    plt.plot(inverse_temps, log_rate_constants, 'o')
    plt.xlabel('1 / Temperature (1/K)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

if __name__ == '__main__':
    main()


Executing /content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/hydrolysis_decomposition_estimates.ipynb...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully executed /content/drive/MyDrive/Colab_Notebooks/MoDuLAAR/hydrolysis_decomposition_estimates.ipynb
Failed to access functions - they may not be defined.
Skipping optimization and plotting due to function access issues.
